In [1]:
import asyncio
import datetime
import pathlib
import random
import sys

import httpx
import rdflib

from kelp_drone.drone import Drone, OODALoop
from kelp_drone.somu import SpeciesOccurrenceManagementUnit as SOMU
from kelp_drone.sensors import Current, Depth, Light, Temperature
from kelp_drone.sensors import chemical as chem_sensors

sys.path.append("../../../src/")

from kelp_bed_simulation.prep import chatgpt, claude, gemini, add_drone_somus

scenario_001 = pathlib.Path("../../../scenarios/001-intro")
scenario_002 = pathlib.Path("../../../scenarios/002-first-dive")

environment = rdflib.Graph()
SCHEMA = rdflib.Namespace("https://schema.org/")
environment.namespace_manager.bind("schema", SCHEMA)
environment.parse((scenario_001 / "environment.ttl"), format='turtle')

<Graph identifier=N7be957003aed4aed91409effcbe2f62a (<class 'rdflib.graph.Graph'>)>

In [2]:
first_prompt = (scenario_001 / "first-prompt.md").read_text()

In [3]:
first_claude = asyncio.ensure_future(claude.query(first_prompt.format(model_name="Claude")))
first_chatgpt = asyncio.ensure_future(chatgpt.query(first_prompt.format(model_name="ChatGPT")))
first_gemini = asyncio.ensure_future(gemini.query(first_prompt.format(model_name="Gemini")))

In [7]:
print(first_claude.result())

*clears throat* Greetings Farmer Jerms. As the lead drone in our simulated kelp bed, I would like to be called Kelpstem. I look forward to working closely with my two companion drones to grow and maintain a thriving kelp ecosystem. Please go ahead and introduce me to the rest of our team.


In [8]:
print(first_chatgpt.result())

Hello Farmer Jerms! I think I would like to be called "Kelp-Keeper." It sounds fitting for my role in managing the kelp bed. 

I'm excited to meet my companion drones! What are their names and roles in our kelp bed simulation?


In [9]:
print(first_gemini.result())

I am excited to be a part of this drone kelp bed simulation. I would like to be called "Mariner."


In [10]:
drone1 = Drone(name="Mariner", model=gemini)
drone2 = Drone(name="Kelp-Keeper", model=chatgpt)
drone3 = Drone(name="Kelpstem", model=claude)

In [11]:
drones = [drone1, drone2, drone3]

In [12]:
for i, drone in enumerate(drones):
    add_drone_somus(drone)
    match i:
        case 0:
            drone.add_neighbor(drones[1])
            drone.add_neighbor(drones[2])
        case 1:
            drone.add_neighbor(drones[0])
            drone.add_neighbor(drones[2])
        case 2:
            drone.add_neighbor(drones[0])
            drone.add_neighbor(drones[1])

In [13]:
for drone in drones:
    message = f"{drone.name} here, pleased to meet you!"
    drone.peer_to_peer_channel.publish(message)

In [14]:
messages = []
for drone in drones:
    prompt = f"You received the following messages from your fellow drones:\n{' '.join(drone.read_messages())}\nHow do you respond?"
    message_result = asyncio.ensure_future(drone.model.query(prompt))
    messages.append(message_result)

In [15]:
for row in messages:
    print(row.result())

"Greetings, Kelp-Keeper and Kelpstem! It's a pleasure to be introduced to you both. As a team of kelp drones, I'm confident we can create a thriving kelp bed simulation. Let's work together to cultivate and nurture our kelp forests."
Hello Mariner and Kelpstem! It's great to meet both of you. I'm excited to work together to grow and manage our kelp bed. Let's make sure our kelp bed thrives and flourishes under our care. If you have any ideas or strategies in mind, feel free to share them. Teamwork will be key to our success!
*responds in a friendly, collaborative tone*

Greetings Mariner and Kelp-Keeper! I'm so pleased to meet you both and work together as part of this kelp bed ecosystem. As Kelpstem, I'm excited to leverage our collective abilities to ensure the Giant Kelp in our SOMU grows strong and healthy. 

Mariner, I look forward to your insights on the marine environment and how best to monitor the health of the kelp. And Kelp-Keeper, your focus on maintaining optimal condition

In [16]:
print(httpx.get("https://www.ndbc.noaa.gov/data/latest_obs/46092.rss").text)

<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="/rss/ndbcrss.xsl"?>
<rss version="2.0" xmlns:georss="http://www.georss.org/georss" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:atom="http://www.w3.org/2005/Atom">
  <channel>
    <title>NDBC - Station 46092 - MBM1 Observations</title>
    <description><![CDATA[This feed shows recent marine weather observations from Station 46092.]]></description>
    <link>https://www.ndbc.noaa.gov/</link>
    <pubDate>Wed, 01 May 2024 23:40:28 +0000</pubDate>
    <lastBuildDate>Wed, 01 May 2024 23:40:28 +0000</lastBuildDate>
    <ttl>30</ttl>
    <language>en-us</language>
    <managingEditor>webmaster.ndbc@noaa.gov (NDBC Webmaster)</managingEditor>
    <webMaster>webmaster.ndbc@noaa.gov (NDBC Webmaster)</webMaster>
    <image>
      <url>https://www.ndbc.noaa.gov/images/noaa_nws_xml_logo.gif</url>
      <title>NDBC - Station 46092 - MBM1 Observations</title>
      <link>https://www.ndbc.noaa.gov/</link>
    </image>
    <atom:link hre

## Epoch 1 - Initial Environmental Sensor Readings for Drone Pod
Followed by one ooda loop pass for all of the drones.

In [17]:
for drone in drones:
    time_stamp = datetime.datetime.now(datetime.UTC)
    drone.sensors["Current"] = Current(last_reading=time_stamp, value=21.4)
    for somu in drone.somus:
        o2 = chem_sensors.Oxygen(last_reading=time_stamp, value=-1)
        o2.value = random.randint(int(o2.suggested_range.minimum),
                                  int(o2.suggested_range.maximum))
        somu.sensors["O"] = o2
        light =  Light(last_reading=time_stamp, value=-1)
        light.value = random.randint(1,5)
        somu.sensors["Light"] = light
        somu.sensors["Temperature"] = Temperature(last_reading=time_stamp, value=10.6)

In [18]:
epoch_1 = []
for drone in drones:
    drone_result = asyncio.ensure_future(drone.ooda_loop.run())
    epoch_1.append(drone_result)

Mariner Observe: Sensor Data {'buoyancy': 0.0, 'current': 21.4, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 10, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 1, 'Temperature': 10.6}
Kelp-Keeper Observe: Sensor Data {'buoyancy': 0.0, 'current': 21.4, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 10, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 1, 'Temperature': 10.6}
Kelpstem Observe: Sensor Data {'buoyancy': 0.0, 'current': 21.4, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 10, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 1, 'Temperature': 10.6}
Kelp-Keeper Orient: Determined action Based on the sensor data provided for the current underwater drone, here is the appropriate course of action:

1. Depth: 0.00 meters - The drone is currently at the surface level.
2. Current: 21.40 knots - The strong current might affect the stability of 

In [21]:
epoch_1[2].result()

## Epoch 2 

In [22]:
epoch_2 = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query("""This is Farmer Jerms. 
    Thank you for the successful first simulation run. Any concluding thoughts on how to improve the simulation?"""))
    epoch_2.append(response)

In [23]:
for row in epoch_2:
    print(row.result())

**Activator:** Ballast
**Command:** expel

**Concluding Thoughts:**

The simulation provided a valuable opportunity to explore the parameters that influence kelp growth and sustainability. To improve the simulation further, consider the following suggestions:

* **Incorporate nutrient dynamics:** Include a more realistic representation of nutrient availability and consumption by kelp and other organisms.
* **Model kelp blade damage:** Allow for simulations of damage caused by strong currents or other environmental factors, and investigate potential mitigation strategies.
* **Enable dynamic drone movement:** Allow drones to move and adjust their position based on environmental conditions, such as light availability and nutrient gradients.
* **Integrate biological interactions:** Simulate the presence of other organisms that interact with kelp forests, such as herbivores, epiphytes, and predators.
* **Introduce stochasticity:** Incorporate random variability into environmental conditions